https://towardsdatascience.com/federated-learning-a-step-by-step-implementation-in-tensorflow-aac568283399
    
    - you can .get_weights() and .set_weights for TF models
    - You can also implement weighted averaging based on, perhaps, how much data the local model saw
    - sending the optimizer states so that we don't have to initialize them again when using .compile() on the local data

In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
def get_samples(num_samples=10): 
    ''' get a random set of sample MNIST images'''
    from tensorflow import keras
    import numpy as np

    # the data, split between train and test sets
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
    
    # take a random set of images
    idx = np.random.choice(np.arange(len(x_train)), num_samples, replace=True)
    x_train = x_train[idx]
    y_train = y_train[idx]
    return x_train, y_train

def get_test_data():
    from tensorflow import keras
    num_classes = 10
    _, (x_test, y_test) = keras.datasets.mnist.load_data()
    x_test = x_test.astype("float32") / 255
    x_test = np.expand_dims(x_test, -1)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    return (x_test, y_test)

In [5]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_test, y_test) = get_test_data()
(x_train, y_train) = get_samples(1000)


# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
#x_test = x_test.astype("float32") / 255

# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
#x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)

global_model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

global_model.summary()

batch_size = 128
epochs = 5

global_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

global_model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

score = global_model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

x_train shape: (1000, 28, 28, 1)
1000 train samples
10000 test samples
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1600)              0         
_________________________________________________

In [6]:
# Local model #1

batch_size = 128
epochs = 5

local_model_1 = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

local_model_1.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# the data, split between train and test sets
(local_x_train_1, Local_y_train_1) = get_samples(100)

# get the global weights
global_weights = global_model.get_weights()

# set the local model's weights to the global ones
local_model_1.set_weights(global_weights)

# train the local model using additional data
local_model_1.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

score = local_model_1.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Epoch 1/5
8/8 [==============================] - 1s 52ms/step - loss: 0.8142 - accuracy: 0.7467 - val_loss: 0.6754 - val_accuracy: 0.7700
Epoch 2/5
8/8 [==============================] - 0s 35ms/step - loss: 0.6568 - accuracy: 0.7978 - val_loss: 0.5215 - val_accuracy: 0.8500
Epoch 3/5
8/8 [==============================] - 0s 32ms/step - loss: 0.5567 - accuracy: 0.8333 - val_loss: 0.4563 - val_accuracy: 0.8700
Epoch 4/5
8/8 [==============================] - 0s 34ms/step - loss: 0.4719 - accuracy: 0.8656 - val_loss: 0.4263 - val_accuracy: 0.8800
Epoch 5/5
8/8 [==============================] - 0s 34ms/step - loss: 0.4439 - accuracy: 0.8600 - val_loss: 0.4117 - val_accuracy: 0.8700
Test loss: 0.3767952024936676
Test accuracy: 0.8906999826431274


In [10]:
# Local model #2

batch_size = 128
epochs = 5

local_model_2 = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

local_model_2.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# the data, split between train and test sets
(local_x_train_2, Local_y_train_2) = get_samples(100)

# get the global weights
global_weights = global_model.get_weights()

# set the local model's weights to the global ones
local_model_2.set_weights(global_weights)

# train the local model using additional data
local_model_2.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

score = local_model_2.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Epoch 1/5
8/8 [==============================] - 1s 50ms/step - loss: 0.7863 - accuracy: 0.7678 - val_loss: 0.6205 - val_accuracy: 0.8500
Epoch 2/5
8/8 [==============================] - 0s 35ms/step - loss: 0.6129 - accuracy: 0.8100 - val_loss: 0.5269 - val_accuracy: 0.8300
Epoch 3/5
8/8 [==============================] - 0s 36ms/step - loss: 0.5275 - accuracy: 0.8489 - val_loss: 0.4688 - val_accuracy: 0.8400
Epoch 4/5
8/8 [==============================] - 0s 36ms/step - loss: 0.4855 - accuracy: 0.8556 - val_loss: 0.4315 - val_accuracy: 0.8700
Epoch 5/5
8/8 [==============================] - 0s 34ms/step - loss: 0.4245 - accuracy: 0.8733 - val_loss: 0.4083 - val_accuracy: 0.8900
Test loss: 0.37870416045188904
Test accuracy: 0.890999972820282


In [9]:
weights_1 = local_model_1.get_weights()
weights_1_np = np.asarray(weights_1, dtype=object)

In [11]:
# getting weights of both models
weights_1 = local_model_1.get_weights()
weights_2 = local_model_2.get_weights()

In [12]:
weights_1_np = np.asarray(weights_1, dtype=object)
weights_2_np = np.asarray(weights_2, dtype=object)

In [13]:
average_weights = np.mean([weights_1_np, weights_2_np], axis=0)

In [14]:
test_global_model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

test_global_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

test_global_model.set_weights(average_weights)

score = test_global_model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.36944931745529175
Test accuracy: 0.8942000269889832


In [15]:
score = global_model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.7627089619636536
Test accuracy: 0.8087000250816345
